In [1]:
import pickle
import gzip
import numpy as np

In [2]:
f = gzip.open("./data/mnist.pkl.gz", "rb")

In [3]:
train_tuple_data, valid_tuple_data, test_tuple_data = pickle.load(f, encoding="bytes")

In [4]:
train_data = train_tuple_data[0]
train_label = train_tuple_data[1]

In [5]:
one_hot_train_label = []
for label in train_label:
    one_hot_label = np.zeros([10])
    one_hot_label[label] = 1
    one_hot_train_label.append(one_hot_label)

In [6]:
valid_data = valid_tuple_data[0]
valid_label = valid_tuple_data[1]

In [7]:
test_data = test_tuple_data[0]
test_label = test_tuple_data[1]

In [8]:
#定义网络
net = [784, 40, 10]
num_layers = len(net)
weights = [np.random.randn(i, j) for i, j in zip(net[:-1], net[1:])]
bias = [np.random.randn(i) for i in net[1:]]

In [9]:
def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [10]:
def cost_derivative(output_activations, y):
    return (output_activations - y)

In [11]:
def forward(batch_data):
    a = batch_data
    for w, b in zip(weights, bias):
        z = np.matmul(a, w) + b
        a = sigmoid(z)
    return a

In [12]:
def back_prop(batch_data, batch_label):
    zs = []
    activates = [batch_data]
    #前向传播
    for w, b in zip(weights, bias):
        z = np.matmul(activates[-1], w) + b
        zs.append(z)
        activates.append(sigmoid(z))
    
    #delta的值
    delta = cost_derivative(activates[-1], batch_label) * sigmoid_derivative(zs[-1])
    #填充位，用于放参数
    nalba_w = [np.zeros(w.shape) for w in weights]
    nalba_b = [np.zeros(b.shape) for b in bias]
    nalba_b[-1] = delta
    nalba_w[-1] =  np.matmul(activates[-2].transpose(), delta)
    for l in range(2, len_net):
        sp = sigmoid_derivative(zs[-l])
        delta = np.matmul(delta, weights[-l+1].transpose()) * sp
        nalba_b[-l] = delta
        nalba_w[-l] = np.matmul(activates[-l-1].transpose(), delta)
    return nalba_w, nalba_b

In [13]:
epoch_num = 30
for epoch in range(epoch_num):
    batch_size = 10
    batch_num = len(train_data)//batch_size
    for batch in range(batch_num):
        begin = batch * batch_size
        end = begin + batch_size
        batch_data = train_data[begin:end]
        batch_label = one_hot_train_label[begin:end]

        nalba_w, nalba_b = back_prop(batch_data, batch_label)
        #跟新w,b
        eta = 3.0
        weights = [w - (eta * nw / batch_size) for w, nw in zip(weights, nalba_w)]
        bias = [b - (eta*np.sum(nb, axis=0) / batch_size) for b, nb in zip(bias, nalba_b)]
    
    x = forward(test_data)
    y = np.argmax(x, axis=1)
    correct_num = np.sum(y == test_label)
    print("epoch:{0}\tcorret:{1}/{2}".format(epoch, correct_num, len(test_data)))

NameError: name 'len_net' is not defined